In [1]:
# 데이터 불러오기
from dotenv import load_dotenv
import pandas as pd
import os 
import numpy as np
from sqlalchemy import create_engine


In [2]:
# .env 파일에서 환경 부르기
load_dotenv()

# 환경 변수 사용
database = os.getenv("CON_STR")


# # csv 파일 경로
# csv_file_path = os.getenv("CSV_FILE_PATH")
# print(csv_file_path)

engine = create_engine(database)

In [8]:
sql1= ''' 
select * from cx.agg_TEREA_Sourcing
'''

gg = pd.read_sql_query(sql=sql1, con=engine)

In [7]:
gg.to_excel('Terea_sourcing_raw2.xlsx')

KeyboardInterrupt: 

In [3]:
# -- Arbor_sourcing_M1
# 직전 3개월 데이터 구매 이력 소싱 - raw data
sql = ''' 
select 
	t.YYYYMM, t.id, t.gender, t.age, 
	b.cigatype,
	b.company,
    b.ProductFamilyCode,
    b.productSubFamilyCode,
	b.New_FLAVORSEG,
    b.New_TARSEGMENTAT,
	b.engname,
    sum(a.pack_qty) qty
from  cx.agg_TEREA_Sourcing t
	join cx.fct_K7_Monthly a on t.id = a.id 
		and a.YYYYMM BETWEEN CONVERT(NVARCHAR(6), DATEADD(MONTH, -3, t.YYYYMM+'01'), 112)
					 AND CONVERT(NVARCHAR(6), DATEADD(MONTH, -1, t.YYYYMM+'01'), 112)	
join cx.product_master_temp b on a.product_code = b.PROD_ID and CIGADEVICE =  'CIGARETTES' AND  cigatype != 'CSV' AND 4 < LEN(a.id)
where t.YYYYMM='202211'	
group by 
	t.YYYYMM, t.id, t.gender, t.age,
	b.cigatype,
	b.company,
    b.ProductFamilyCode,
    b.productSubFamilyCode,
	b.New_FLAVORSEG,
    b.New_TARSEGMENTAT,
	b.engname
'''

data = pd.read_sql_query(sql=sql, con=engine)


In [4]:
data

,YYYYMM,id,gender,age,cigatype,company,ProductFamilyCode,productSubFamilyCode,New_FLAVORSEG,New_TARSEGMENTAT,engname,qty
0,202211,003DE9124299CC396950FEF717EFE5020EFCB079E8DECB...,남,40대,CC,BAT,DUNHILL,DUNHILLKS,Regular,ULT,DUNHILL 3MG,1.0
1,202211,003DE9124299CC396950FEF717EFE5020EFCB079E8DECB...,남,40대,HnB,BAT,GLO,NEOSTICKS,Regular,None,NEOSTIKS BRIGHT TOBACCO,31.0
2,202211,007BE5AEC24991C378435D650C80E7C168971700CB8EDF...,남,40대,HnB,PMK,IQOS,HEETS,Fresh,None,HEETS BLUE LABEL,3.0
3,202211,007BE5AEC24991C378435D650C80E7C168971700CB8EDF...,남,40대,HnB,PMK,IQOS,HEETS,New Taste,None,HEETS SUMMER BREEZE,1.0
4,202211,00851229FF4A0026F2682594CEDABB0AE1B73FF85E6CDE...,남,50대,HnB,PMK,IQOS,HEETS,New Taste,None,HEETS BLACK PURPLE SELECTION,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...
3295,202211,FFEA5E5C9C441901DFBBAA534D9EF782B50A7B88D52B6B...,여,50대,CC,KTG,RAISON,RASIONFR,New Taste,1MG,RAISON FRENCH YOGO,1.0
3296,202211,FFEA5E5C9C441901DFBBAA534D9EF782B50A7B88D52B6B...,여,50대,CC,KTG,THEONE,THEONE,Regular,1MG,THE ONE BLUE,1.0
3297,202211,FFEA5E5C9C441901DFBBAA534D9EF782B50A7B88D52B6B...,여,50대,CC,PMK,MLB,MLBREG,Regular,LTS,MARLBORO GOLD ORIGINAL,1.0
3298,202211,FFEA5E5C9C441901DFBBAA534D9EF782B50A7B88D52B6B...,여,50대,HnB,BAT,GLO,NEOSTICKS,Fresh,None,NEOSTIKS FRESH MIX,1.0


In [10]:
# -- Arbor_sourcing_M1 작업

data['company'] = data['company'].apply(lambda x : x.strip())

result = data.groupby(['YYYYMM', 'id', 'gender', 'age']).agg({
    'company': lambda x: x.unique(),
    'New_FLAVORSEG': lambda x: x.unique()
})


# result.set_index(['YYYYMM', 'id', 'gender', 'age'])
result

company  \
YYYYMM id                                                 gender age                    
202211 003DE9124299CC396950FEF717EFE5020EFCB079E8DECB8... 남      40대            [BAT]   
       007BE5AEC24991C378435D650C80E7C168971700CB8EDFD... 남      40대            [PMK]   
       00851229FF4A0026F2682594CEDABB0AE1B73FF85E6CDED... 남      50대            [PMK]   
       00FA9CFB5FFA3F6E3C00DA693D6862019C5A42D6A6D0B35... 남      30대            [KTG]   
       01117EF5EFBB1B6D53108D3EEBF53FAFBECFA132F880CFB... 남      30대       [KTG, PMK]   
...                                                                               ...   
       FF83E54787EB82C6293ADA53E369F68215B39162BD33E3E... 남      50대            [PMK]   
       FFC50D9BE8D22E040EDAE5B54B00FA3856C2240DE21BE4C... 남      20대            [KTG]   
       FFC7C4D03CF065384860A5176E832123A07E70B6E185396... 여      50대            [KTG]   
       FFD0E8F349A14622CD91B697DA35E0E9D4FFAF7D9D4C4EA... 남      40대            [KTG]   
       FFEA5E5C9C441901DFBBAA534D9EF782B50A7B88D52B6BA... 여      50대  [KTG, PMK, BAT]   

                                                                                    New_FLAVORSEG  
YYYYMM id                                                 gender age                               
202211 003DE9124299CC396950FEF717EFE5020EFCB079E8DECB8... 남      40대                    [Regular]  
       007BE5AEC24991C378435D650C80E7C168971700CB8EDFD... 남      40대           [Fresh, New Taste]  
       00851229FF4A0026F2682594CEDABB0AE1B73FF85E6CDED... 남      50대         [New Taste, Regular]  
       00FA9CFB5FFA3F6E3C00DA693D6862019C5A42D6A6D0B35... 남      30대                  [New Taste]  
       01117EF5EFBB1B6D53108D3EEBF53FAFBECFA132F880CFB... 남      30대           [New Taste, Fresh]  
...                                                                                           ...  
       FF83E54787EB82C6293ADA53E369F68215B39162BD33E3E... 남      50대             [Fresh, Regular]  
       FFC50D9BE8D22E040EDAE5B54B00FA3856C2240DE21BE4C... 남      20대                      [Fresh]  
       FFC7C4D03CF065384860A5176E832123A07E70B6E185396... 여      50대                      [Fresh]  
       FFD0E8F349A14622CD91B697DA35E0E9D4FFAF7D9D4C4EA... 남      40대                    [Regular]  
       FFEA5E5C9C441901DFBBAA534D9EF782B50A7B88D52B6BA... 여      50대  [New Taste, Regular, Fresh]  

[1418 rows x 2 columns]

In [13]:
pivot_table1 = data.pivot_table(
    index=['YYYYMM', 'id', 'gender', 'age'] ,           # 행 인덱스
    columns=['company'],
    values=['qty'],
    aggfunc='sum'            
)

pivot_table1


qty  \
company                                                                BAT   
YYYYMM id                                                 gender age         
202211 003DE9124299CC396950FEF717EFE5020EFCB079E8DECB8... 남      40대  32.0   
       007BE5AEC24991C378435D650C80E7C168971700CB8EDFD... 남      40대   NaN   
       00851229FF4A0026F2682594CEDABB0AE1B73FF85E6CDED... 남      50대   NaN   
       00FA9CFB5FFA3F6E3C00DA693D6862019C5A42D6A6D0B35... 남      30대   NaN   
       01117EF5EFBB1B6D53108D3EEBF53FAFBECFA132F880CFB... 남      30대   NaN   
...                                                                    ...   
       FF83E54787EB82C6293ADA53E369F68215B39162BD33E3E... 남      50대   NaN   
       FFC50D9BE8D22E040EDAE5B54B00FA3856C2240DE21BE4C... 남      20대   NaN   
       FFC7C4D03CF065384860A5176E832123A07E70B6E185396... 여      50대   NaN   
       FFD0E8F349A14622CD91B697DA35E0E9D4FFAF7D9D4C4EA... 남      40대   NaN   
       FFEA5E5C9C441901DFBBAA534D9EF782B50A7B88D52B6BA... 여      50대   1.0   

                                                                               \
company                                                              JTI  KTG   
YYYYMM id                                                 gender age            
202211 003DE9124299CC396950FEF717EFE5020EFCB079E8DECB8... 남      40대 NaN  NaN   
       007BE5AEC24991C378435D650C80E7C168971700CB8EDFD... 남      40대 NaN  NaN   
       00851229FF4A0026F2682594CEDABB0AE1B73FF85E6CDED... 남      50대 NaN  NaN   
       00FA9CFB5FFA3F6E3C00DA693D6862019C5A42D6A6D0B35... 남      30대 NaN  9.0   
       01117EF5EFBB1B6D53108D3EEBF53FAFBECFA132F880CFB... 남      30대 NaN  2.0   
...                                                                   ..  ...   
       FF83E54787EB82C6293ADA53E369F68215B39162BD33E3E... 남      50대 NaN  NaN   
       FFC50D9BE8D22E040EDAE5B54B00FA3856C2240DE21BE4C... 남      20대 NaN  1.0   
       FFC7C4D03CF065384860A5176E832123A07E70B6E185396... 여      50대 NaN  4.0   
       FFD0E8F349A14622CD91B697DA35E0E9D4FFAF7D9D4C4EA... 남      40대 NaN  2.0   
       FFEA5E5C9C441901DFBBAA534D9EF782B50A7B88D52B6BA... 여      50대 NaN  9.0   

                                                                            
company                                                                PMK  
YYYYMM id                                                 gender age        
202211 003DE9124299CC396950FEF717EFE5020EFCB079E8DECB8... 남      40대   NaN  
       007BE5AEC24991C378435D650C80E7C168971700CB8EDFD... 남      40대   4.0  
       00851229FF4A0026F2682594CEDABB0AE1B73FF85E6CDED... 남      50대   7.0  
       00FA9CFB5FFA3F6E3C00DA693D6862019C5A42D6A6D0B35... 남      30대   NaN  
       01117EF5EFBB1B6D53108D3EEBF53FAFBECFA132F880CFB... 남      30대  13.0  
...                                                                    ...  
       FF83E54787EB82C6293ADA53E369F68215B39162BD33E3E... 남      50대  10.0  
       FFC50D9BE8D22E040EDAE5B54B00FA3856C2240DE21BE4C... 남      20대   NaN  
       FFC7C4D03CF065384860A5176E832123A07E70B6E185396... 여      50대   NaN  
       FFD0E8F349A14622CD91B697DA35E0E9D4FFAF7D9D4C4EA... 남      40대   NaN  
       FFEA5E5C9C441901DFBBAA534D9EF782B50A7B88D52B6BA... 여      50대   1.0  

[1418 rows x 4 columns]

In [12]:
data['CC_taste'] = data['cigatype'] + ' ' + data['New_FLAVORSEG']

pivot_table2 = data.pivot_table(
    index=['YYYYMM', 'id', 'gender', 'age'] ,
    columns=['CC_taste'],
    values=['qty'],  
    aggfunc='sum'           
)

pivot_table2

qty  \
CC_taste                                                             CC Fresh   
YYYYMM id                                                 gender age            
202211 003DE9124299CC396950FEF717EFE5020EFCB079E8DECB8... 남      40대      NaN   
       007BE5AEC24991C378435D650C80E7C168971700CB8EDFD... 남      40대      NaN   
       00851229FF4A0026F2682594CEDABB0AE1B73FF85E6CDED... 남      50대      NaN   
       00FA9CFB5FFA3F6E3C00DA693D6862019C5A42D6A6D0B35... 남      30대      NaN   
       01117EF5EFBB1B6D53108D3EEBF53FAFBECFA132F880CFB... 남      30대      NaN   
...                                                                       ...   
       FF83E54787EB82C6293ADA53E369F68215B39162BD33E3E... 남      50대      NaN   
       FFC50D9BE8D22E040EDAE5B54B00FA3856C2240DE21BE4C... 남      20대      NaN   
       FFC7C4D03CF065384860A5176E832123A07E70B6E185396... 여      50대      4.0   
       FFD0E8F349A14622CD91B697DA35E0E9D4FFAF7D9D4C4EA... 남      40대      NaN   
       FFEA5E5C9C441901DFBBAA534D9EF782B50A7B88D52B6BA... 여      50대      NaN   

                                                                                   \
CC_taste                                                             CC New Taste   
YYYYMM id                                                 gender age                
202211 003DE9124299CC396950FEF717EFE5020EFCB079E8DECB8... 남      40대          NaN   
       007BE5AEC24991C378435D650C80E7C168971700CB8EDFD... 남      40대          NaN   
       00851229FF4A0026F2682594CEDABB0AE1B73FF85E6CDED... 남      50대          NaN   
       00FA9CFB5FFA3F6E3C00DA693D6862019C5A42D6A6D0B35... 남      30대          9.0   
       01117EF5EFBB1B6D53108D3EEBF53FAFBECFA132F880CFB... 남      30대          1.0   
...                                                                           ...   
       FF83E54787EB82C6293ADA53E369F68215B39162BD33E3E... 남      50대          NaN   
       FFC50D9BE8D22E040EDAE5B54B00FA3856C2240DE21BE4C... 남      20대          NaN   
       FFC7C4D03CF065384860A5176E832123A07E70B6E185396... 여      50대          NaN   
       FFD0E8F349A14622CD91B697DA35E0E9D4FFAF7D9D4C4EA... 남      40대          NaN   
       FFEA5E5C9C441901DFBBAA534D9EF782B50A7B88D52B6BA... 여      50대          3.0   

                                                                                 \
CC_taste                                                             CC Regular   
YYYYMM id                                                 gender age              
202211 003DE9124299CC396950FEF717EFE5020EFCB079E8DECB8... 남      40대        1.0   
       007BE5AEC24991C378435D650C80E7C168971700CB8EDFD... 남      40대        NaN   
       00851229FF4A0026F2682594CEDABB0AE1B73FF85E6CDED... 남      50대        NaN   
       00FA9CFB5FFA3F6E3C00DA693D6862019C5A42D6A6D0B35... 남      30대        NaN   
       01117EF5EFBB1B6D53108D3EEBF53FAFBECFA132F880CFB... 남      30대        NaN   
...                                                                         ...   
       FF83E54787EB82C6293ADA53E369F68215B39162BD33E3E... 남      50대        NaN   
       FFC50D9BE8D22E040EDAE5B54B00FA3856C2240DE21BE4C... 남      20대        NaN   
       FFC7C4D03CF065384860A5176E832123A07E70B6E185396... 여      50대        NaN   
       FFD0E8F349A14622CD91B697DA35E0E9D4FFAF7D9D4C4EA... 남      40대        2.0   
       FFEA5E5C9C441901DFBBAA534D9EF782B50A7B88D52B6BA... 여      50대        5.0   

                                                                                \
CC_taste                                                             HnB Fresh   
YYYYMM id                                                 gender age             
202211 003DE9124299CC396950FEF717EFE5020EFCB079E8DECB8... 남      40대       NaN   
       007BE5AEC24991C378435D650C80E7C168971700CB8EDFD... 남      40대       3.0   
       00851229FF4A0026F2682594CEDABB0AE1B73FF85E6CDED... 남      50대       NaN   
       00FA9CFB5FFA3F6E3C00DA693D6862019C5A42D6A6D0B35... 남      30대       NaN   
       0111

In [17]:

result =data[data['productSubFamilyCode'].isin(['AIIM', 'FIIT', 'HEETS', 'MIIX', 'NEO', 'NEOSTICKS', 'TEREA'])]

data['HnB_taste'] = result['productSubFamilyCode'] + ' ' + data['New_FLAVORSEG']

pivot_table3 = data.pivot_table(
    index=['YYYYMM', 'id', 'gender', 'age'] ,            
    columns=['HnB_taste'],
    values=['qty'],  
    aggfunc='sum'            
)

pivot_table3

qty  \
HnB_taste                                                            FIIT Fresh   
YYYYMM id                                                 gender age              
202211 003DE9124299CC396950FEF717EFE5020EFCB079E8DECB8... 남      40대        NaN   
       007BE5AEC24991C378435D650C80E7C168971700CB8EDFD... 남      40대        NaN   
       00851229FF4A0026F2682594CEDABB0AE1B73FF85E6CDED... 남      50대        NaN   
       01117EF5EFBB1B6D53108D3EEBF53FAFBECFA132F880CFB... 남      30대        NaN   
       018BDE68F71093A6A0E7C55C8FC9F17632BD1BB32EA6A5C... 남      40대        NaN   
...                                                                         ...   
       FF6D169BC2951D794CF9AF897DDA79D48423425D5053220... 남      50대        NaN   
       FF8008FF518BE7144371D9250B3EEF6A021601FC869E9AB... 남      40대        NaN   
       FF83E54787EB82C6293ADA53E369F68215B39162BD33E3E... 남      50대        NaN   
       FFC50D9BE8D22E040EDAE5B54B00FA3856C2240DE21BE4C... 남      20대        NaN   
       FFEA5E5C9C441901DFBBAA534D9EF782B50A7B88D52B6BA... 여      50대        NaN   

                                                                                     \
HnB_taste                                                            FIIT New Taste   
YYYYMM id                                                 gender age                  
202211 003DE9124299CC396950FEF717EFE5020EFCB079E8DECB8... 남      40대            NaN   
       007BE5AEC24991C378435D650C80E7C168971700CB8EDFD... 남      40대            NaN   
       00851229FF4A0026F2682594CEDABB0AE1B73FF85E6CDED... 남      50대            NaN   
       01117EF5EFBB1B6D53108D3EEBF53FAFBECFA132F880CFB... 남      30대            1.0   
       018BDE68F71093A6A0E7C55C8FC9F17632BD1BB32EA6A5C... 남      40대            NaN   
...                                                                             ...   
       FF6D169BC2951D794CF9AF897DDA79D48423425D5053220... 남      50대            NaN   
       FF8008FF518BE7144371D9250B3EEF6A021601FC869E9AB... 남      40대            NaN   
       FF83E54787EB82C6293ADA53E369F68215B39162BD33E3E... 남      50대            NaN   
       FFC50D9BE8D22E040EDAE5B54B00FA3856C2240DE21BE4C... 남      20대            NaN   
       FFEA5E5C9C441901DFBBAA534D9EF782B50A7B88D52B6BA... 여      50대            NaN   

                                                                                   \
HnB_taste                                                            FIIT Regular   
YYYYMM id                                                 gender age                
202211 003DE9124299CC396950FEF717EFE5020EFCB079E8DECB8... 남      40대          NaN   
       007BE5AEC24991C378435D650C80E7C168971700CB8EDFD... 남      40대          NaN   
       00851229FF4A0026F2682594CEDABB0AE1B73FF85E6CDED... 남      50대          NaN   
       01117EF5EFBB1B6D53108D3EEBF53FAFBECFA132F880CFB... 남      30대          NaN   
       018BDE68F71093A6A0E7C55C8FC9F17632BD1BB32EA6A5C... 남      40대          NaN   
...                                                                           ...   
       FF6D169BC2951D794CF9AF897DDA79D48423425D5053220... 남      50대          NaN   
       FF8008FF518BE7144371D9250B3EEF6A021601FC869E9AB... 남      40대          NaN   
       FF83E54787EB82C6293ADA53E369F68215B39162BD33E3E... 남      50대          NaN   
       FFC50D9BE8D22E040EDAE5B54B00FA3856C2240DE21BE4C... 남      20대          NaN   
       FFEA5E5C9C441901DFBBAA534D9EF782B50A7B88D52B6BA... 여      50대          NaN   

                                                                                  \
HnB_taste                                                            HEETS Fresh   
YYYYMM id                                                 gender age               
202211 003DE9124299CC396950FEF717EFE5020EFCB079E8DECB8... 남      40대         NaN   
       007BE5AEC24991C378435D650C80E7C168971700CB8EDFD... 남      40대         3.0   
       00851229FF4A0026F2682594CEDABB0AE1B73FF85E6CDED... 남      50대         NaN   

In [ ]:
filtered_data = data[data['ProductFamilyCode'] == 'IQOS'][['engname']]
filtered_data['engname'].unique()

In [ ]:
filtered_data = data[data['ProductFamilyCode'] == 'IQOS']

pivot_table4 = filtered_data.pivot_table(
    index=['YYYYMM', 'id', 'gender', 'age'] ,           
    columns=['engname'],
    values=['qty'],   
    aggfunc='sum'           
)

pivot_table4

In [18]:
df2 = pd.DataFrame(result).set_index(['YYYYMM', 'id', 'gender', 'age'])

combined = pd.concat([result, pivot_table1, pivot_table2, pivot_table3, pivot_table4], axis=1, keys=['YYYYMM', 'id', 'gender', 'age'] )

combined.sort_index(level='id')
# combined.to_excel('finish.xlsx')
combined

C:\Users\schoi43\AppData\Local\Temp\ipykernel_13424\2587795094.py:3: FutureWarning: The behavior of pd.concat with len(keys) != len(objs) is deprecated. In a future version this will raise instead of truncating to the smaller of the two sequences
  combined = pd.concat([result, pivot_table1, pivot_table2, pivot_table3, pivot_table4], axis=1, keys=['YYYYMM', 'id', 'gender', 'age'] )


AssertionError: Cannot concat indices that do not have the same number of levels

In [14]:
# 필터링된 데이터 생성
filtered_data = data[data['ProductFamilyCode'] == 'IQOS']
 
# 피벗 테이블 생성
pivot_table4 = filtered_data.pivot_table(index=['YYYYMM', 'id', 'gender', 'age'],
                                         columns=['engname'], values=['qty'],
                                         aggfunc='sum')
 
# HnB_taste 컬럼 생성 및 피벗 테이블 생성
result = data[data['productSubFamilyCode'].isin(['AIIM', 'FIIT', 'HEETS', 'MIIX', 'NEO', 'NEOSTICKS', 'TEREA'])]
data['HnB_taste'] = result['productSubFamilyCode'] + ' ' + data['New_FLAVORSEG']

pivot_table3 = data.pivot_table(index=['YYYYMM', 'id', 'gender', 'age'],
                                columns=['HnB_taste'], values=['qty'],
                                aggfunc='sum')
 
# CC_taste 컬럼 생성 및 피벗 테이블 생성
data['CC_taste'] = data['cigatype'] + ' ' + data['New_FLAVORSEG']

pivot_table2 = data.pivot_table(index=['YYYYMM', 'id', 'gender', 'age'],
                                columns=['CC_taste'], values=['qty'],
                                aggfunc='sum')
 
# 기본 피벗 테이블 생성
pivot_table1 = data.pivot_table(index=['YYYYMM', 'id', 'gender', 'age'],
                                columns=['company'], values=['qty'],
                                aggfunc='sum')

# 피벗 테이블들을 하나의 데이터프레임으로 병합
concatenated_pivot = pd.concat([ pivot_table1, pivot_table2, pivot_table3, pivot_table4], axis=1)

# 결과 출력
# final = pd.merge(result, concatenated_pivot, how='outer')
concatenated_pivot

qty  \
                                                                       BAT   
YYYYMM id                                                 gender age         
202211 003DE9124299CC396950FEF717EFE5020EFCB079E8DECB8... 남      40대  32.0   
       007BE5AEC24991C378435D650C80E7C168971700CB8EDFD... 남      40대   NaN   
       00851229FF4A0026F2682594CEDABB0AE1B73FF85E6CDED... 남      50대   NaN   
       00FA9CFB5FFA3F6E3C00DA693D6862019C5A42D6A6D0B35... 남      30대   NaN   
       01117EF5EFBB1B6D53108D3EEBF53FAFBECFA132F880CFB... 남      30대   NaN   
...                                                                    ...   
       FF83E54787EB82C6293ADA53E369F68215B39162BD33E3E... 남      50대   NaN   
       FFC50D9BE8D22E040EDAE5B54B00FA3856C2240DE21BE4C... 남      20대   NaN   
       FFC7C4D03CF065384860A5176E832123A07E70B6E185396... 여      50대   NaN   
       FFD0E8F349A14622CD91B697DA35E0E9D4FFAF7D9D4C4EA... 남      40대   NaN   
       FFEA5E5C9C441901DFBBAA534D9EF782B50A7B88D52B6BA... 여      50대   1.0   

                                                                               \
                                                                     JTI  KTG   
YYYYMM id                                                 gender age            
202211 003DE9124299CC396950FEF717EFE5020EFCB079E8DECB8... 남      40대 NaN  NaN   
       007BE5AEC24991C378435D650C80E7C168971700CB8EDFD... 남      40대 NaN  NaN   
       00851229FF4A0026F2682594CEDABB0AE1B73FF85E6CDED... 남      50대 NaN  NaN   
       00FA9CFB5FFA3F6E3C00DA693D6862019C5A42D6A6D0B35... 남      30대 NaN  9.0   
       01117EF5EFBB1B6D53108D3EEBF53FAFBECFA132F880CFB... 남      30대 NaN  2.0   
...                                                                   ..  ...   
       FF83E54787EB82C6293ADA53E369F68215B39162BD33E3E... 남      50대 NaN  NaN   
       FFC50D9BE8D22E040EDAE5B54B00FA3856C2240DE21BE4C... 남      20대 NaN  1.0   
       FFC7C4D03CF065384860A5176E832123A07E70B6E185396... 여      50대 NaN  4.0   
       FFD0E8F349A14622CD91B697DA35E0E9D4FFAF7D9D4C4EA... 남      40대 NaN  2.0   
       FFEA5E5C9C441901DFBBAA534D9EF782B50A7B88D52B6BA... 여      50대 NaN  9.0   

                                                                            \
                                                                       PMK   
YYYYMM id                                                 gender age         
202211 003DE9124299CC396950FEF717EFE5020EFCB079E8DECB8... 남      40대   NaN   
       007BE5AEC24991C378435D650C80E7C168971700CB8EDFD... 남      40대   4.0   
       00851229FF4A0026F2682594CEDABB0AE1B73FF85E6CDED... 남      50대   7.0   
       00FA9CFB5FFA3F6E3C00DA693D6862019C5A42D6A6D0B35... 남      30대   NaN   
       01117EF5EFBB1B6D53108D3EEBF53FAFBECFA132F880CFB... 남      30대  13.0   
...                                                                    ...   
       FF83E54787EB82C6293ADA53E369F68215B39162BD33E3E... 남      50대  10.0   
       FFC50D9BE8D22E040EDAE5B54B00FA3856C2240DE21BE4C... 남      20대   NaN   
       FFC7C4D03CF065384860A5176E832123A07E70B6E185396... 여      50대   NaN   
       FFD0E8F349A14622CD91B697DA35E0E9D4FFAF7D9D4C4EA... 남      40대   NaN   
       FFEA5E5C9C441901DFBBAA534D9EF782B50A7B88D52B6BA... 여      50대   1.0   

                                                                               \
                                                                     CC Fresh   
YYYYMM id                                                 gender age            
202211 003DE9124299CC396950FEF717EFE5020EFCB079E8DECB8... 남      40대      NaN   
       007BE5AEC24991C378435D650C80E7C168971700CB8EDFD... 남      40대      NaN   
       00851229FF4A0026F2682594CEDABB0AE1B73FF85E6CDED... 남      50대      NaN   
       00FA9CFB5FFA3F6E3C00DA693D6862019C5A42D6A6D0B35... 남      30대      NaN   
       01117EF5EFBB1B6D53108D3EEBF53FAFBECFA132F880CFB... 남      30대      NaN   
...                                                                       ...   
       FF83E54787EB82

In [16]:
total_id_count = data.groupby('YYYYMM')['id'].count().reset_index(name='total_id_count')

gender_age_count = data.groupby(['YYYYMM', 'gender', 'age']).size().reset_index(name='count')

gender_pivot = gender_age_count.pivot_table(
    index ='YYYYMM',
    columns= ['gender', 'age'],
    values='count'
)

res = total_id_count.set_index('YYYYMM').join(gender_pivot)

res

MergeError: Not allowed to merge between different levels. (1 levels on the left, 2 on the right)